In [ ]:
! pip install -U spacy -q

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.3                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines        en_core_web_lg (3.2.0)        



In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

Dataset: resumes sampled from https://github.com/florex/resume_corpus and annotated manually


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 6.4 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import json
f = open('/content/drive//MyDrive/hfg ner/annotations_all_competency.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

 18%|█▊        | 34/193 [00:00<00:02, 56.68it/s]

Skipping entity


100%|██████████| 193/193 [00:01<00:00, 106.55it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy -F

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
#!rm -rf model-best
#!rm -rf model-last
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy 


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-03-14 06:16:21,881] [INFO] Set up nlp object from config
[2022-03-14 06:16:21,893] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-14 06:16:21,898] [INFO] Created vocabulary
[2022-03-14 06:16:27,038] [INFO] Added vectors: en_core_web_lg
tcmalloc: large alloc 1643593728 bytes == 0x559601d9c000 @  0x7f23421672a4 0x7f2336ce28bd 0x7f2336ce1a9d 0x7f2336cde275 0x7f2336cdea1e 0x5595bd4e81cd 0x5595bd5dab3d 0x5595bd55c458 0x5595bd55702f 0x5595bd4e9aba 0x5595bd558cd4 0x7f21b63cd1c8 0x7f21b63cf16e 0x7f21b63d5ca7 0x7f21b63d8f32 0x5595bd4e8b32 0x5595bd55c1dd 0x5595bd4e99da 0x5595bd557eae 0x7f21b63cd1c8 0x7f21b63cf16e 0x7f21b63d5980 0x5595bd4e8424 0x5595bd4e8120 0x5595bd55cb80 0x5595bd55766e 0x5595bd4e9aba 0x5595bd558cd4 0x5595bd55766e 0x5595bd4e9aba 0x5595bd558cd4
tcmalloc: large alloc 1662509056 bytes == 0x559695ed4000 @  0x7f23421672a4 0x7f2336cdffd3 0x7f2336ce1a9d 0x7f233

terms.txt database from https://github.com/nicolamelluso/SkillNER

Using entity_ruler in conjunction with NER

In [ ]:
with open('/content/drive/MyDrive/hfg ner/terms.txt') as f:
  terms = f.readlines()
patterns = []
terms[0].split(' ')
for i in terms:
  tokens = i.rstrip().split(' ')
  tokens_new = [{"LOWER":j} for j in tokens]
  patterns.append({"label":"SOFT_SKILL","pattern":tokens_new})

In [ ]:
#from spacy.lang.en import English
nlp_ner = spacy.load("/content/new_model/model-best")
#nlp_ner = spacy.load("en_core_web_lg", exclude=["ner"])
ruler = nlp_ner.add_pipe("entity_ruler", before = "ner")
ruler.add_patterns(patterns)

In [ ]:
text = "I believe that I will be a valuable asset due to my leadership abilities, my problem-solving skills, and my ability to adapt quickly to fast-paced environments. I can communicate in an effective and supportive manner."

In [ ]:
doc=nlp_ner(text)

In [ ]:
spacy.displacy.render(doc,style ="ent", jupyter=True)

In [ ]:
text_2 = "In my interactions with Peggy, I have found that she exhibits great resilience, SQL tenacity and motivation. Throughout our workshops, she showed herself to be a quick learner who is eager to learn. When facing difficulties, she stays calm and collected. I am competent with SQL and accounting. C# C++ Python Java database, public relations, marketing"

In [ ]:
doc=nlp_ner(text_2)

In [ ]:
spacy.displacy.render(doc,style ="ent", jupyter=True)

In [ ]:
!zip -r /content/model-best.zip /content/model-best

  adding: content/model-best/ (stored 0%)
  adding: content/model-best/config.cfg (deflated 60%)
  adding: content/model-best/meta.json (deflated 58%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/vocab/key2row (deflated 16%)
  adding: content/model-best/vocab/strings.json (deflated 78%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 8%)
  adding: content/model-best/tokenizer (deflated 80%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/moves (deflated 63%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/model (deflated 7%)
  adding: content/model-best/tok2vec/ (stored 0%)
  adding: content/model-best/tok2vec/cfg (stored 0%)
  adding: content/model-best/tok2vec/model (deflated 7%)


In [ ]:
!pip install streamlit

     |████████████████████████████████| 9.9 MB 2.6 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 181 kB 57.3 MB/s 
     |████████████████████████████████| 164 kB 57.6 MB/s 
     |████████████████████████████████| 4.3 MB 31.2 MB/s 
     |████████████████████████████████| 111 kB 63.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 128 kB 49.0 MB/s 
     |████████████████████████████████| 793 kB 44.5 MB/s 
     |████████████████████████████████| 380 kB 26.3 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=7c031fa92d0b16a6e90373bf2b7e6c81e21b5af259fd710b55d114ee100bc723
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import spacy
import base64
from PIL import Image
from spacy import displacy

HTML_WRAPPER = """<div style="overflow-x: auto; border: 1px solid #e6e9ef; border-radius: 0.25rem; padding: 1rem; 
                margin-bottom: 2.5rem">{}</div>"""

@st.cache(allow_output_mutation=True)


def load_model():
    with open('/content/drive/MyDrive/hfg ner/terms.txt') as f:
      terms = f.readlines()
    patterns = []
    terms[0].split(' ')
    for i in terms:
      tokens = i.rstrip().split(' ')
      tokens_new = [{"LOWER":j} for j in tokens]
      patterns.append({"label":"SOFT_SKILL","pattern":tokens_new})
    nlp_ner = spacy.load("/content/drive/MyDrive/hfg ner/model-best")
    ruler = nlp_ner.add_pipe("entity_ruler", before = "ner")
    ruler.add_patterns(patterns)
    return nlp_ner

@st.cache()
def run_nlp(text,model):
    doc = model(text)
    html = displacy.render(doc, style="ent")
    html = html.replace("\n", " ")
    st.write(HTML_WRAPPER.format(html), unsafe_allow_html=True)




# img= Image.open("logo.jpeg")
# st.sidebar.image(img, width=300)

st.header("hack for good 2022")
st.caption("Aloysius Chow, Megan Yee, Natalie Chung, Justin Lim")
st.title("Welcome.")

skills_type = st.radio(
     "What skills would you like to detect?",
     ('Soft Skills', 'Technical Skills'))
nlp = load_model()


if skills_type == 'Soft Skills':
    colors = {"SOFT_SKILL": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
    options = {"ents": ["SOFT_SKILL"], "colors": colors}
    answer_1 = st.text_area("How do you approach problems? What is your process?", height=10)
    doc1 = nlp(answer_1)
    html1 = displacy.render(doc1, style="ent", options = options)
    html1 = html1.replace("\n", " ")
    st.write(HTML_WRAPPER.format(html1), unsafe_allow_html=True)
    answer_2 = st.text_area("Give an example of when you worked well in a team.", height=10)
    doc2 = nlp(answer_2)
    html2 = displacy.render(doc2, style="ent", options=options)
    html2 = html2.replace("\n", " ")
    st.write(HTML_WRAPPER.format(html2), unsafe_allow_html=True)
    answer_3 = st.text_area("What is the most challenging/difficult situation you’ve ever had to resolve, and how did you do it?", height=10)
    doc3 = nlp(answer_3)
    html3 = displacy.render(doc3, style="ent", options=options)
    html3 = html3.replace("\n", " ")
    st.write(HTML_WRAPPER.format(html3), unsafe_allow_html=True)

    
elif skills_type == 'Technical Skills':
    data = st.file_uploader("Upload your CV here", type=['pdf', 'docx'])

    if data is not None:
        pass

st.button("Generate Skills", on_click=run_nlp())

Overwriting app.py
